In [2]:
!pip install finance-datareader
!pip install pykrx
!pip install --upgrade pandas_ta
!pip install "numpy<1.24"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=69aa8caa81ccb5fb96afe6831f99a988be9ba1547fb8b5d256ecdada8e8d23b4
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: numpy


In [ ]:
import FinanceDataReader as fdr
from pykrx import stock
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
import os
import time
import warnings
from datetime import datetime, timedelta

# 경고 메시지 무시 (pandas_ta 등에서 발생 가능)
warnings.filterwarnings('ignore')

# --- 1. 설정값 정의 ---
START_DATE = "20210101" # 데이터 시작일 (최소 1년 이상 권장, 예: 2년)
END_DATE = "20250430"   # 데이터 종료일
MARKETS_TO_PROCESS = ["KOSPI", "KOSDAQ"] # 처리할 시장 ("KONEX" 추가 가능)

# 저장 경로 설정 (Google Drive)
DRIVE_MOUNT_PATH = '/content/drive'
BASE_SAVE_PATH = '/content/drive/MyDrive/processed_stock_data_full_v1' # 최종 저장 경로 (버전 관리 추천)
RAW_DATA_PATH = os.path.join(BASE_SAVE_PATH, 'raw')       # 원본 데이터 저장 경로 (선택 사항)
PROCESSED_DATA_PATH = os.path.join(BASE_SAVE_PATH, 'processed_parquet') # 최종 처리 데이터 저장 경로
SCALER_SAVE_PATH = os.path.join(BASE_SAVE_PATH, 'scalers') # 스케일러 저장 경로 (선택 사항)

# 기술적 지표 설정
SMA_LENGTHS = [5, 20, 60, 120]
RSI_LENGTH = 14
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
BBANDS_LENGTH = 20
BBANDS_STD = 2
ATR_LENGTH = 14
STOCH_K = 14
STOCH_D = 3

# 정규화 대상 컬럼 (OHLCV + 거래대금 + 기술지표 + 재무/거시 지표 등)
# 여기에 포함되지 않은 컬럼 (예: is_month_end 등 바이너리 값)은 정규화하지 않음
# 실제 모델링 시 피처 중요도에 따라 선택 필요
COLUMNS_TO_NORMALIZE = [
    'open', 'high', 'low', 'close', 'volume', 'amount', # 기본 OHLCV, 거래대금
    *[f'SMA_{l}' for l in SMA_LENGTHS], f'RSI_{RSI_LENGTH}', # SMA, RSI
    f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', # MACD
    f'BBL_{BBANDS_LENGTH}_{BBANDS_STD}.0', f'BBM_{BBANDS_LENGTH}_{BBANDS_STD}.0', f'BBU_{BBANDS_LENGTH}_{BBANDS_STD}.0', f'BBB_{BBANDS_LENGTH}_{BBANDS_STD}.0', f'BBP_{BBANDS_LENGTH}_{BBANDS_STD}.0', # Bollinger Bands
    f'ATRr_{ATR_LENGTH}', # ATR
    'OBV', # OBV
    f'STOCHk_{STOCH_K}_{STOCH_D}_3', f'STOCHd_{STOCH_K}_{STOCH_D}_3', # Stochastic
    'PBR', 'PER', # 재무
    'USD_KRW' # 거시
]

# --- 2. Google Drive 마운트 및 경로 생성 ---
print("--- Google Drive 마운트 및 경로 생성 ---")
try:
    from google.colab import drive
    drive.mount(DRIVE_MOUNT_PATH)
    print("Google Drive 마운트 완료.")
except ImportError:
    print("Google Colab 환경이 아닙니다. 로컬 경로를 사용합니다.")
    BASE_SAVE_PATH = './processed_stock_data_full_v1' # 로컬 경로 예시
    RAW_DATA_PATH = os.path.join(BASE_SAVE_PATH, 'raw')
    PROCESSED_DATA_PATH = os.path.join(BASE_SAVE_PATH, 'processed_parquet')
    SCALER_SAVE_PATH = os.path.join(BASE_SAVE_PATH, 'scalers')

os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
os.makedirs(SCALER_SAVE_PATH, exist_ok=True)
print(f"데이터 저장 기본 경로: {BASE_SAVE_PATH}")
print(f"처리된 데이터 저장 경로: {PROCESSED_DATA_PATH}")
print(f"스케일러 저장 경로: {SCALER_SAVE_PATH}")

# --- 3. 전역 데이터 수집 (거시경제, 재무) ---
print("\n--- 전역 데이터 수집 (거시경제, 재무) ---")
global_fetch_start = time.time()

# 3.1 거시경제 데이터 (USD/KRW 환율)
print("  > USD/KRW 환율 데이터 수집...")
try:
    df_usdkrw = fdr.DataReader('USD/KRW', START_DATE, END_DATE)[['Close']]
    df_usdkrw.rename(columns={'Close': 'USD_KRW'}, inplace=True)
    df_usdkrw.index = pd.to_datetime(df_usdkrw.index.date) # 날짜 형식 통일
    print(f"    - 환율 데이터 수집 완료: {df_usdkrw.shape[0]} 행")
except Exception as e:
    print(f"    - 오류: 환율 데이터 수집 실패 - {e}")
    df_usdkrw = pd.DataFrame(columns=['USD_KRW']) # 빈 데이터프레임 생성

# 3.2 재무 데이터 (PBR, PER 등) - pykrx 사용
# 주의: 이 부분은 시간이 매우 오래 걸리거나 API 제한에 걸릴 수 있습니다.
print(f"  > 재무 데이터 수집 ({START_DATE}~{END_DATE})... (시간 소요)")
all_fundamental_data = {}
all_tickers_list = []
for market in MARKETS_TO_PROCESS:
    all_tickers_list.extend(stock.get_market_ticker_list(date=END_DATE, market=market))

# 날짜 범위를 생성하여 각 날짜별로 재무 데이터 조회 시도
# 현실적으로 모든 날짜 조회는 매우 느리므로, 월말 기준으로 조회하는 등의 최적화 필요
# 여기서는 요청대로 전체 기간 조회 시도 (매우 느릴 수 있음)
date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
df_fundamental_daily = pd.DataFrame()

# 최적화: 날짜별로 모든 티커의 재무 정보를 가져오는 것은 매우 비효율적.
# 월말 또는 분기말 기준으로 가져오는 것이 현실적임.
# 예시: 월말 기준으로 변경
monthly_dates = pd.date_range(start=START_DATE, end=END_DATE, freq='M').strftime('%Y%m%d').tolist()
print(f"  > 재무 데이터 월말 기준({len(monthly_dates)} 개)으로 수집 시도...")
fundamental_dfs = []
for date_str in monthly_dates:
    try:
        # pykrx는 영업일이 아니면 가장 가까운 과거 영업일 기준으로 데이터를 가져옴
        df_fund_part = stock.get_market_fundamental(date_str) # 모든 시장 한번에
        df_fund_part['date'] = pd.to_datetime(date_str) # 날짜 컬럼 추가
        fundamental_dfs.append(df_fund_part[['date', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']])
        if len(fundamental_dfs) % 12 == 0: # 1년치 모일 때마다 로그
             print(f"    - {date_str} 까지 재무 데이터 수집 중...")
        time.sleep(0.5) # API 부하 방지
    except Exception as e:
        print(f"    - 경고: {date_str} 재무 데이터 수집 중 오류 - {e}")
        continue

if fundamental_dfs:
    df_fundamental_full = pd.concat(fundamental_dfs)
    df_fundamental_full.reset_index(inplace=True) # ticker가 인덱스 -> 컬럼으로
    df_fundamental_full.rename(columns={'티커': 'ticker'}, inplace=True)
    df_fundamental_full.set_index(['date', 'ticker'], inplace=True)
    df_fundamental_full = df_fundamental_full[~df_fundamental_full.index.duplicated(keep='last')] # 중복 제거
    print(f"    - 재무 데이터 (월말 기준) 처리 완료: {df_fundamental_full.shape[0]} 레코드")
else:
    print("    - 재무 데이터 수집 실패 또는 데이터 없음.")
    df_fundamental_full = pd.DataFrame()


global_fetch_end = time.time()
print(f"  > 전역 데이터 수집 총 소요 시간: {(global_fetch_end - global_fetch_start)/60:.2f} 분")


# --- 4. 개별 종목 데이터 처리 및 저장 루프 ---
print("\n--- 개별 종목 데이터 처리 및 저장 시작 ---")
total_process_start_time = time.time()
processed_count = 0
failed_count = 0

# 전체 티커 리스트 (중복 제거)
all_tickers_unique = sorted(list(set(all_tickers_list)))
print(f"총 {len(all_tickers_unique)}개 종목 처리 시작...")

for i, ticker in enumerate(all_tickers_unique):
    print(f"\n[{i+1}/{len(all_tickers_unique)}] Ticker: {ticker} 처리 시작...")
    ticker_start_time = time.time()
    individual_scalers = {} # 이 종목의 스케일러 저장용

    try:
        # 4.1 OHLCV 데이터 가져오기
        df_ohlcv = stock.get_market_ohlcv(START_DATE, END_DATE, ticker)
        if df_ohlcv.empty:
            print(f"  >> 데이터 없음. 건너뜁니다.")
            failed_count += 1
            continue

        # 4.2 거래대금 데이터 가져오기
        df_amount = stock.get_market_trading_value_by_date(START_DATE, END_DATE, ticker)[['기관합계', '기타법인', '개인', '외국인합계']] # 예시: 투자자별 거래대금
        df_amount['amount'] = df_amount.sum(axis=1) # 총 거래대금 (근사치)

        # 데이터 병합 (OHLCV + 거래대금)
        df_stock = pd.merge(df_ohlcv, df_amount[['amount']], left_index=True, right_index=True, how='left')
        df_stock.index.name = 'date' # 인덱스 이름 설정
        df_stock.index = pd.to_datetime(df_stock.index.date) # 날짜 형식 통일

        # 4.3 컬럼 이름 영문 변경
        df_stock.rename(columns={
            '시가': 'open', '고가': 'high', '저가': 'low', '종가': 'close',
            '거래량': 'volume', '등락률': 'change' # 등락률도 포함
        }, inplace=True)

        # 4.4 기술적 지표 추가
        print("  > 기술적 지표 계산...")
        # SMA
        for length in SMA_LENGTHS:
            df_stock.ta.sma(length=length, append=True)
        # RSI
        df_stock.ta.rsi(length=RSI_LENGTH, append=True)
        # MACD
        df_stock.ta.macd(fast=MACD_FAST, slow=MACD_SLOW, signal=MACD_SIGNAL, append=True)
        # Bollinger Bands
        df_stock.ta.bbands(length=BBANDS_LENGTH, std=BBANDS_STD, append=True)
        # ATR
        df_stock.ta.atr(length=ATR_LENGTH, append=True)
        # OBV
        df_stock.ta.obv(append=True)
        # Stochastic
        df_stock.ta.stoch(k=STOCH_K, d=STOCH_D, append=True)

        # 4.5 전역 데이터 병합 (재무, 거시)
        print("  > 전역 데이터 병합...")
        # 거시 데이터 병합
        df_stock = pd.merge(df_stock, df_usdkrw, left_index=True, right_index=True, how='left')
        df_stock['USD_KRW'].ffill(inplace=True) # 주말/휴일 등으로 누락된 환율 채우기

        # 재무 데이터 병합
        if not df_fundamental_full.empty:
            # 현재 티커에 해당하는 재무 데이터만 필터링
            if ticker in df_fundamental_full.index.get_level_values('ticker'):
                 df_fund_ticker = df_fundamental_full[df_fundamental_full.index.get_level_values('ticker') == ticker]
                 df_fund_ticker = df_fund_ticker.droplevel('ticker') # ticker 레벨 제거
                 # 인덱스를 date로 맞춰서 merge_asof (가장 가까운 과거 값 사용)
                 df_stock = pd.merge_asof(df_stock.sort_index(),
                                          df_fund_ticker[['PBR', 'PER']].sort_index(),
                                          left_index=True, right_index=True,
                                          direction='backward')
            else: # 해당 티커의 재무 데이터가 없는 경우
                df_stock['PBR'] = np.nan
                df_stock['PER'] = np.nan
        else: # 전체 재무 데이터가 없는 경우
            df_stock['PBR'] = np.nan
            df_stock['PER'] = np.nan

        # 재무 데이터 ffill (월말/분기말 데이터를 해당 기간 동안 유효하다고 가정)
        df_stock[['PBR', 'PER']].ffill(inplace=True)

        # 4.6 이벤트 특징 추가
        print("  > 이벤트 특징 추가...")
        df_stock['is_month_end'] = df_stock.index.is_month_end.astype(int)
        # 추가적인 이벤트 특징 (예: 요일, 월초 등) 필요시 여기에 추가

        # 4.7 최종 데이터 정리 (NaN 처리)
        print("  > 최종 데이터 정리 (NaN 처리)...")
        initial_rows = len(df_stock)
        # 기술적 지표 계산 등으로 초기에 발생하는 NaN 제거
        df_stock.dropna(subset=COLUMNS_TO_NORMALIZE, how='any', inplace=True) # 정규화 대상 컬럼 기준 NaN 제거
        # 또는 전체 NaN 포함 행 제거: df_stock.dropna(inplace=True)
        final_rows = len(df_stock)
        print(f"    - NaN 처리 후 데이터: {final_rows} 행 (처리 전: {initial_rows})")

        if final_rows < 60: # 너무 짧으면 의미 없다고 판단 (임의 기준)
            print(f"  >> 최종 데이터 길이 부족 ({final_rows}). 건너<0xEB><0x8A>니다.")
            failed_count += 1
            continue

        # 4.8 데이터 정규화 (MinMaxScaler)
        print("  > 데이터 정규화...")
        df_normalized = df_stock.copy()
        scalers = {} # 이 종목의 스케일러 저장

        # 정규화 대상 컬럼들에 대해 스케일링 적용
        for col in COLUMNS_TO_NORMALIZE:
            if col in df_normalized.columns:
                # NaN 값이 있는지 다시 한번 확인 (ffill 후에도 남을 수 있음)
                if df_normalized[col].isnull().any():
                    print(f"    - 경고: 정규화 전 '{col}' 컬럼에 NaN 발견. ffill 시도.")
                    df_normalized[col].ffill(inplace=True)
                    df_normalized[col].bfill(inplace=True) # 맨 앞 NaN 처리 위해 bfill도 적용
                # 모든 값이 동일한 경우 스케일링 시 오류 발생 가능
                if df_normalized[col].nunique() > 1:
                     scaler = MinMaxScaler()
                     # fit_transform은 2D 배열을 기대하므로 reshape 사용
                     df_normalized[col + '_norm'] = scaler.fit_transform(df_normalized[[col]])
                     scalers[col] = scaler # 스케일러 저장 (나중에 inverse_transform 위해)
                else:
                     # 모든 값이 같으면 0 또는 0.5 등으로 스케일링 (상황에 맞게)
                     df_normalized[col + '_norm'] = 0.0
                     print(f"    - 경고: '{col}' 컬럼의 모든 값이 동일하여 0으로 정규화.")

            else:
                print(f"    - 경고: 정규화 대상 컬럼 '{col}'이 데이터프레임에 없습니다.")

        # (선택) 스케일러 객체 저장 (joblib 사용 추천)
        # import joblib
        # scaler_filename = os.path.join(SCALER_SAVE_PATH, f'{ticker}_scalers.joblib')
        # joblib.dump(scalers, scaler_filename)

        # 4.9 최종 데이터 파일 저장 (Parquet)
        print("  > 최종 데이터 Parquet 파일 저장...")
        # 정규화된 컬럼과 원본 컬럼, 플래그 컬럼 등을 모두 포함하여 저장
        final_save_path = os.path.join(PROCESSED_DATA_PATH, f"{ticker}.parquet")
        df_normalized.to_parquet(final_save_path)
        print(f"    - 저장 완료: {final_save_path}")
        processed_count += 1

    except Exception as e:
        print(f"  >> 오류: 티커 {ticker} 처리 중 예외 발생: {e}")
        import traceback
        traceback.print_exc()
        failed_count += 1
        continue
    finally:
        ticker_end_time = time.time()
        print(f"  Ticker: {ticker} 처리 시간: {ticker_end_time - ticker_start_time:.2f} 초")


total_process_end_time = time.time()
print("\n--- 전체 개별 종목 데이터 처리 및 저장 완료 ---")
print(f"총 소요 시간: {(total_process_end_time - total_process_start_time) / 60:.2f} 분")
print(f"성공적으로 처리/저장된 종목 수: {processed_count}")
print(f"실패 또는 건너뛴 종목 수: {failed_count}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/005500.parquet
  Ticker: 005500 처리 시간: 4.26 초

[298/2758] Ticker: 005610 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 944 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/005610.parquet
  Ticker: 005610 처리 시간: 4.33 초

[299/2758] Ticker: 005670 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 005670 처리 시간: 3.48 초

[300/2758] Ticker: 005680 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 944 행 (처리 전: 1063)
  > 데이터 

Traceback (most recent call last):
  File "<ipython-input-1-56b04594ab5a>", line 230, in <cell line: 0>
    df_stock.dropna(subset=COLUMNS_TO_NORMALIZE, how='any', inplace=True) # 정규화 대상 컬럼 기준 NaN 제거
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 6670, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['SMA_60', 'SMA_120']


  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 031310 처리 시간: 3.72 초

[745/2758] Ticker: 031330 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 031330 처리 시간: 3.89 초

[746/2758] Ticker: 031430 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 941 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/031430.parquet
  Ticker: 031430 처리 시간: 6.61 초

[747/2758] Ticker: 031440 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 944 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parq

Traceback (most recent call last):
  File "<ipython-input-1-56b04594ab5a>", line 230, in <cell line: 0>
    df_stock.dropna(subset=COLUMNS_TO_NORMALIZE, how='any', inplace=True) # 정규화 대상 컬럼 기준 NaN 제거
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 6670, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['SMA_120']


  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 064480 처리 시간: 3.20 초

[1162/2758] Ticker: 064520 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 064520 처리 시간: 3.01 초

[1163/2758] Ticker: 064550 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 064550 처리 시간: 3.36 초

[1164/2758] Ticker: 064760 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 064760 처리 시간: 3.44 초

[1165/2758] Ticker: 064800 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 

...

  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 102120 처리 시간: 3.31 초

[1545/2758] Ticker: 102260 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 944 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/102260.parquet
  Ticker: 102260 처리 시간: 4.10 초

[1546/2758] Ticker: 102280 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 501 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
    - 경고: 'PER' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/102280.parquet
  Ticker: 102280 처리 시간: 4.03 초

[1547/2758] Ticker: 102370 처리 시작...
 

Traceback (most recent call last):
  File "<ipython-input-1-56b04594ab5a>", line 230, in <cell line: 0>
    df_stock.dropna(subset=COLUMNS_TO_NORMALIZE, how='any', inplace=True) # 정규화 대상 컬럼 기준 NaN 제거
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 6670, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['SMA_120']


  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 944 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/163560.parquet
  Ticker: 163560 처리 시간: 4.16 초

[1783/2758] Ticker: 163730 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1044)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 163730 처리 시간: 3.08 초

[1784/2758] Ticker: 166090 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 166090 처리 시간: 3.33 초

[1785/2758] Ticker: 166480 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 166480 처리 시간: 3.26 초


Traceback (most recent call last):
  File "<ipython-input-1-56b04594ab5a>", line 230, in <cell line: 0>
    df_stock.dropna(subset=COLUMNS_TO_NORMALIZE, how='any', inplace=True) # 정규화 대상 컬럼 기준 NaN 제거
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 6670, in dropna
    raise KeyError(np.array(subset)[check].tolist())
KeyError: ['SMA_120']


  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 178320 처리 시간: 3.39 초

[1809/2758] Ticker: 178780 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 178780 처리 시간: 2.84 초

[1810/2758] Ticker: 178920 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 898 행 (처리 전: 1063)
  > 데이터 정규화...
    - 경고: 'amount' 컬럼의 모든 값이 동일하여 0으로 정규화.
  > 최종 데이터 Parquet 파일 저장...
    - 저장 완료: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet/178920.parquet
  Ticker: 178920 처리 시간: 4.10 초

[1811/2758] Ticker: 179290 처리 시작...
  > 기술적 지표 계산...
  > 전역 데이터 병합...
  > 이벤트 특징 추가...
  > 최종 데이터 정리 (NaN 처리)...
    - NaN 처리 후 데이터: 0 행 (처리 전: 1063)
  >> 최종 데이터 길이 부족 (0). 건너<0xEB><0x8A>니다.
  Ticker: 179290 처리 시간: 3.50 초


In [ ]:
import pandas as pd
import os

# --- 설정: Parquet 파일이 저장된 경로 ---
# 이전 데이터 처리 코드에서 사용한 PROCESSED_DATA_PATH와 동일하게 설정
parquet_directory = '/content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet'

print(f"확인할 Parquet 파일 경로: {parquet_directory}")

try:
    # 해당 디렉토리의 파일 목록 가져오기
    files = os.listdir(parquet_directory)
    # Parquet 파일만 필터링 (.parquet 확장자)
    parquet_files = [f for f in files if f.endswith('.parquet')]

    if not parquet_files:
        print(f"오류: '{parquet_directory}' 경로에 Parquet 파일이 없습니다. 경로를 확인해주세요.")
    else:
        # 목록에서 첫 번째 Parquet 파일 선택 (어떤 파일이든 구조는 동일해야 함)
        example_file_name = parquet_files[0]
        example_file_path = os.path.join(parquet_directory, example_file_name)
        print(f"\n샘플 파일 선택: {example_file_name}")

        # 선택한 Parquet 파일 읽기
        df_sample = pd.read_parquet(example_file_path)

        # 컬럼 이름 목록 가져와서 출력
        column_names = df_sample.columns.tolist()
        print("\n컬럼 이름 목록:")
        # 보기 좋게 한 줄에 여러 개씩 출력
        for i in range(0, len(column_names), 5):
             print(column_names[i:i+5])

        # (선택) 데이터 샘플 확인 (처음 3행)
        # print("\n데이터 샘플 (처음 3행):")
        # print(df_sample.head(3))

except FileNotFoundError:
    print(f"오류: '{parquet_directory}' 경로를 찾을 수 없습니다. Google Drive가 마운트되었는지, 경로가 정확한지 확인해주세요.")
except Exception as e:
    print(f"파일 읽기 또는 처리 중 오류 발생: {e}")

확인할 Parquet 파일 경로: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet

샘플 파일 선택: 000020.parquet

컬럼 이름 목록:
['open', 'high', 'low', 'close', 'volume']
['change', 'amount', 'SMA_5', 'SMA_20', 'SMA_60']
['SMA_120', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9']
['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0']
['ATRr_14', 'OBV', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'USD_KRW']
['PBR', 'PER', 'is_month_end', 'open_norm', 'high_norm']
['low_norm', 'close_norm', 'volume_norm', 'amount_norm', 'SMA_5_norm']
['SMA_20_norm', 'SMA_60_norm', 'SMA_120_norm', 'RSI_14_norm', 'MACD_12_26_9_norm']
['MACDs_12_26_9_norm', 'MACDh_12_26_9_norm', 'BBL_20_2.0_norm', 'BBM_20_2.0_norm', 'BBU_20_2.0_norm']
['BBB_20_2.0_norm', 'BBP_20_2.0_norm', 'ATRr_14_norm', 'OBV_norm', 'STOCHk_14_3_3_norm']
['STOCHd_14_3_3_norm', 'PBR_norm', 'PER_norm', 'USD_KRW_norm']


In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os
import time
import tensorflow as tf # TensorFlow 임포트 확인
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import warnings
import gc # 가비지 컬렉션 명시적 호출용

warnings.filterwarnings('ignore') # 경고 무시

# --- 1. 설정값 정의 ---
DRIVE_DATA_PATH = '/content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet' # ★ 데이터 경로 (이전 단계에서 저장한 경로)
DRIVE_MODEL_SAVE_PATH = '/content/drive/MyDrive/lstm_models_per_stock_v1' # ★ 모델 저장 경로 (이전과 동일 또는 새 버전)

# 모델 하이퍼파라미터
TIME_STEPS = 20
TRAIN_RATIO = 0.7
VALID_RATIO = 0.15
EPOCHS = 50 # 최대 에포크 (EarlyStopping 사용)
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# ★★★ 사용할 피처 컬럼 목록 (사용자가 제공한 컬럼 기반 업데이트) ★★★
FEATURE_COLUMNS_NORM = [
    'open_norm', 'high_norm', 'low_norm', 'close_norm', 'volume_norm', 'amount_norm', # 기본 OHLCV 정규화
    'SMA_5_norm', 'SMA_20_norm', 'SMA_60_norm', 'SMA_120_norm', # 이동평균선 정규화
    'RSI_14_norm', # RSI 정규화
    'MACD_12_26_9_norm', 'MACDs_12_26_9_norm', 'MACDh_12_26_9_norm', # MACD 관련 정규화
    'BBL_20_2.0_norm', 'BBM_20_2.0_norm', 'BBU_20_2.0_norm', 'BBB_20_2.0_norm', 'BBP_20_2.0_norm', # 볼린저밴드 정규화
    'ATRr_14_norm', # ATR 정규화
    'OBV_norm', # OBV 정규화
    'STOCHk_14_3_3_norm', 'STOCHd_14_3_3_norm', # 스토캐스틱 정규화
    'PBR_norm', 'PER_norm', # 재무지표 정규화
    'USD_KRW_norm', # 거시경제지표 정규화
    'is_month_end' # 이벤트 피처 (정규화 안 된 원본값 사용 예시)
]
TARGET_COLUMN_NORM = 'close_norm' # 예측 대상 컬럼 (정규화된 종가)

LIMIT_STOCKS = None # ★★★ None으로 두면 전체(918개) 학습, 테스트 시 5 등으로 설정 ★★★

# --- 2. 경로 설정 및 Google Drive 마운트 확인 ---
print("--- 경로 설정 및 Google Drive 확인 ---")
try:
    from google.colab import drive
    drive.mount('/content/drive') # 마운트 확인 또는 재시도
except ImportError:
    print("Google Colab 환경 아님. 로컬 경로 사용 가정.")
    # 로컬 경로 재설정 (필요시)
    # DRIVE_DATA_PATH = 'path/to/your/local/processed_data'
    # DRIVE_MODEL_SAVE_PATH = 'path/to/your/local/models'

os.makedirs(DRIVE_MODEL_SAVE_PATH, exist_ok=True)
print(f"처리된 데이터 경로: {DRIVE_DATA_PATH}")
print(f"모델 저장 경로: {DRIVE_MODEL_SAVE_PATH}")

# --- 3. 처리할 데이터 파일 목록 가져오기 ---
try:
    processed_files = [f for f in os.listdir(DRIVE_DATA_PATH) if f.endswith('.parquet')]
    if not processed_files:
        raise FileNotFoundError(f"처리된 Parquet 파일이 '{DRIVE_DATA_PATH}' 경로에 없습니다.")
    # 파일 목록 정렬 (선택적, 순서 보장)
    processed_files.sort()
    print(f"총 {len(processed_files)}개의 처리된 데이터 파일 확인.")
    if LIMIT_STOCKS is not None and LIMIT_STOCKS > 0:
        processed_files = processed_files[:LIMIT_STOCKS]
        print(f"★★ 테스트를 위해 {LIMIT_STOCKS}개 종목만 처리합니다. ★★")
except FileNotFoundError as e:
    print(f"오류: {e}. 데이터 경로를 확인하세요.")
    exit()
except Exception as e:
    print(f"파일 목록 로딩 중 오류 발생: {e}")
    exit()

# --- 4. 시퀀스 생성 함수 ---
def create_sequences(data, target, time_steps=10):
    Xs, ys = [], []
    feature_data = data
    for i in range(len(feature_data) - time_steps):
        Xs.append(feature_data[i:(i + time_steps)])
        ys.append(target[i + time_steps])
    return np.array(Xs), np.array(ys)

# --- 5. LSTM 모델 정의 함수 ---
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=32, activation='relu')) # 활성화 함수 추가 (옵션)
    model.add(Dense(units=1)) # 최종 출력층
    return model

# --- 6. 전체 종목 모델 학습 루프 ---
print(f"\n--- {len(processed_files)}개 종목 모델 학습 시작 ---")
total_start_time = time.time()
models_trained = 0
models_failed = 0

for i, file_name in enumerate(processed_files):
    ticker = file_name.replace('.parquet', '')
    file_path = os.path.join(DRIVE_DATA_PATH, file_name)
    print(f"\n[{i+1}/{len(processed_files)}] Ticker: {ticker} 학습 시작...")
    ticker_start_time = time.time()

    try:
        # 6.1 데이터 로드
        df_loaded = pd.read_parquet(file_path)

        # 사용할 피처 컬럼만 선택 (존재 여부 확인)
        available_features = [col for col in FEATURE_COLUMNS_NORM if col in df_loaded.columns]
        missing_features = [col for col in FEATURE_COLUMNS_NORM if col not in available_features]
        if missing_features:
            print(f"  >> 경고: 일부 피처 컬럼 누락: {missing_features}. 사용 가능한 피처만 사용합니다.")
            if len(available_features) == 0:
                 print(f"  >> 오류: 사용 가능한 피처가 없습니다. 건너<0xEB><0x8A>니다.")
                 models_failed += 1
                 continue

        if TARGET_COLUMN_NORM not in df_loaded.columns:
             print(f"  >> 오류: 타겟 컬럼 '{TARGET_COLUMN_NORM}'이 없습니다. 건너<0xEB><0x8A>니다.")
             models_failed += 1
             continue

        # 피처 및 타겟 데이터 추출
        feature_data = df_loaded[available_features].values
        target_data = df_loaded[TARGET_COLUMN_NORM].values

        # NaN 값 확인 (Parquet 저장 시 NaN이 없어야 하지만, 로드 후 확인)
        if np.isnan(feature_data).any() or np.isnan(target_data).any():
             print(f"  >> 경고: 피처 또는 타겟 데이터에 NaN 존재. nan_to_num 처리 시도.")
             # NaN을 0으로 대체 (다른 전략 고려 가능: 평균값 대체 등)
             feature_data = np.nan_to_num(feature_data, nan=0.0)
             target_data = np.nan_to_num(target_data, nan=0.0)
             # 필요시 is_month_end 등 특정 컬럼은 별도 처리

        # 6.2 시퀀스 생성
        X, y = create_sequences(feature_data, target_data, TIME_STEPS)

        if len(X) < (TIME_STEPS * 2): # 분할에 충분한 시퀀스 길이 확인
             print(f"  >> 경고: 시퀀스 생성 후 데이터 길이 부족 ({len(X)}). 건너<0xEB><0x8A>니다.")
             models_failed += 1
             continue

        # 6.3 데이터 분할 (Train/Validation/Test)
        n_total = len(X)
        n_train = int(n_total * TRAIN_RATIO)
        n_valid = int(n_total * VALID_RATIO)
        # n_test = n_total - n_train - n_valid

        X_train, y_train = X[:n_train], y[:n_train]
        X_valid, y_valid = X[n_train:n_train + n_valid], y[n_train:n_train + n_valid]
        X_test, y_test = X[n_train + n_valid:], y[n_train + n_valid:]

        if len(X_train) == 0 or len(X_valid) == 0:
             print(f"  >> 경고: 학습 또는 검증 데이터 부족 (Train: {len(X_train)}, Valid: {len(X_valid)}). 건너<0xEB><0x8A>니다.")
             models_failed += 1
             continue

        print(f"  >> 데이터 분할: Train={len(X_train)}, Validation={len(X_valid)}, Test={len(X_test)}")
        print(f"  >> 입력 데이터 형태 (첫 샘플): {X_train[0].shape}") # (TIME_STEPS, 피처 수) 확인

        # 6.4 모델 구축 및 컴파일
        # 입력 shape은 (TIME_STEPS, 피처 수)
        model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error') # MSE loss

        # 6.5 모델 학습
        model_save_path = os.path.join(DRIVE_MODEL_SAVE_PATH, f"{ticker}.keras")

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', verbose=0, save_best_only=True)

        print(f"  >> 모델 학습 시작 (최대 {EPOCHS} 에포크)...")
        history = model.fit(
            X_train, y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_data=(X_valid, y_valid),
            callbacks=[early_stopping, model_checkpoint],
            verbose=0 # 로그 최소화
        )

        # 6.6 학습 완료 및 결과 요약
        # 학습 기록이 있는지 확인 (EarlyStopping으로 1 epoch만에 끝날 수도 있음)
        if 'val_loss' in history.history and len(history.history['val_loss']) > 0:
             best_epoch = np.argmin(history.history['val_loss']) + 1
             min_val_loss = np.min(history.history['val_loss'])
             print(f"  >> 모델 학습 완료. 최적 에포크: {best_epoch}, 최소 검증 손실(MSE): {min_val_loss:.6f}")
        else:
             print(f"  >> 모델 학습 완료 (기록 없음 - 조기 종료 가능성).")
             min_val_loss = float('inf') # 저장 여부 판단 위해

        # 저장 여부 확인 (ModelCheckpoint는 val_loss가 개선될 때만 저장)
        if os.path.exists(model_save_path):
             print(f"  >> 최적 모델 저장됨: {model_save_path}")
             models_trained += 1
             # (선택) 테스트셋 평가
             # test_loss = model.evaluate(X_test, y_test, verbose=0)
             # print(f"  >> 테스트셋 손실 (MSE): {test_loss:.6f}")
        else:
             # val_loss가 한번도 개선되지 않으면 파일이 저장 안될 수 있음
             print(f"  >> 정보: 검증 손실이 개선되지 않아 최적 모델 파일이 저장되지 않았을 수 있습니다 (최소 val_loss: {min_val_loss:.6f}).")
             # 실패 카운트는 에러 발생 시에만 증가시키므로 여기서는 증가시키지 않음 (학습은 시도했으므로)

        # 메모리 관리
        del df_loaded, feature_data, target_data, X, y, X_train, y_train, X_valid, y_valid, X_test, y_test, model, history
        tf.keras.backend.clear_session()
        gc.collect()

    except Exception as e:
        print(f"  >> 오류: 티커 {ticker} 처리 중 예외 발생: {e}")
        import traceback
        traceback.print_exc()
        models_failed += 1
        continue
    finally:
        ticker_end_time = time.time()
        print(f"  Ticker: {ticker} 처리 시간: {ticker_end_time - ticker_start_time:.2f} 초")


total_end_time = time.time()
print("\n--- 전체 종목 모델 학습 완료 ---")
print(f"총 소요 시간: {(total_end_time - total_start_time) / 60:.2f} 분")
print(f"성공적으로 학습/저장된 모델 수: {models_trained}")
print(f"실패 또는 건너뛴 종목 수: {models_failed}")

--- 경로 설정 및 Google Drive 확인 ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
처리된 데이터 경로: /content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet
모델 저장 경로: /content/drive/MyDrive/lstm_models_per_stock_v1
총 918개의 처리된 데이터 파일 확인.

--- 918개 종목 모델 학습 시작 ---

[1/918] Ticker: 000020 학습 시작...
  >> 데이터 분할: Train=417, Validation=89, Test=90
  >> 입력 데이터 형태 (첫 샘플): (20, 27)
  >> 모델 학습 시작 (최대 50 에포크)...
  >> 모델 학습 완료. 최적 에포크: 9, 최소 검증 손실(MSE): 0.004256
  >> 최적 모델 저장됨: /content/drive/MyDrive/lstm_models_per_stock_v1/000020.keras
  Ticker: 000020 처리 시간: 22.28 초

[2/918] Ticker: 000040 학습 시작...
  >> 데이터 분할: Train=406, Validation=87, Test=88
  >> 입력 데이터 형태 (첫 샘플): (20, 27)
  >> 모델 학습 시작 (최대 50 에포크)...
  >> 모델 학습 완료. 최적 에포크: 15, 최소 검증 손실(MSE): 0.001172
  >> 최적 모델 저장됨: /content/drive/MyDrive/lstm_models_per_stock_v1/000040.keras
  Ticker: 000040 처리 시간: 25.00 초

[3/918] Ticker: 000050 학습 시작...
  >> 데이터 분할: Tra

테스트

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os
import time
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler # Scaler 재생성 위해 필요 (옵션)
import warnings
import gc

warnings.filterwarnings('ignore')

# --- 1. 설정값 정의 (이전 학습 코드와 동일하게 유지) ---
DRIVE_DATA_PATH = '/content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet' # 데이터 경로
DRIVE_MODEL_SAVE_PATH = '/content/drive/MyDrive/lstm_models_per_stock_v1' # ★ 모델이 저장된 경로

# 모델 하이퍼파라미터 (학습 때와 동일하게 설정)
TIME_STEPS = 20
TRAIN_RATIO = 0.7
VALID_RATIO = 0.15
# TEST_RATIO = 1 - TRAIN_RATIO - VALID_RATIO (자동 계산됨)

# 사용할 피처 컬럼 목록 (학습 때와 동일하게 설정)
FEATURE_COLUMNS_NORM = [
    'open_norm', 'high_norm', 'low_norm', 'close_norm', 'volume_norm', 'amount_norm',
    'SMA_5_norm', 'SMA_20_norm', 'SMA_60_norm', 'SMA_120_norm', 'RSI_14_norm',
    'MACD_12_26_9_norm', 'MACDs_12_26_9_norm', 'MACDh_12_26_9_norm',
    'BBL_20_2.0_norm', 'BBM_20_2.0_norm', 'BBU_20_2.0_norm', 'BBB_20_2.0_norm', 'BBP_20_2.0_norm',
    'ATRr_14_norm', 'OBV_norm', 'STOCHk_14_3_3_norm', 'STOCHd_14_3_3_norm',
    'PBR_norm', 'PER_norm', 'USD_KRW_norm', 'is_month_end'
]
TARGET_COLUMN_NORM = 'close_norm'

# 테스트할 티커 목록 (이전 테스트에서 성공한 5개)
# 실제로는 성공적으로 학습된 전체 모델 목록을 대상으로 할 수 있음
TICKERS_TO_EVALUATE = ['000020', '000040', '000050', '000070', '000075']

print(f"--- {len(TICKERS_TO_EVALUATE)}개 종목 모델 테스트셋 평가 시작 ---")

# --- 2. 시퀀스 생성 함수 (이전과 동일) ---
def create_sequences(data, target, time_steps=10):
    Xs, ys = [], []
    feature_data = data
    for i in range(len(feature_data) - time_steps):
        Xs.append(feature_data[i:(i + time_steps)])
        ys.append(target[i + time_steps])
    return np.array(Xs), np.array(ys)

# --- 3. 각 종목별 평가 루프 ---
results = {}
for i, ticker in enumerate(TICKERS_TO_EVALUATE):
    print(f"\n[{i+1}/{len(TICKERS_TO_EVALUATE)}] Ticker: {ticker} 평가 시작...")
    model_path = os.path.join(DRIVE_MODEL_SAVE_PATH, f"{ticker}.keras")
    data_path = os.path.join(DRIVE_DATA_PATH, f"{ticker}.parquet")

    try:
        # 3.1 모델 로드
        if not os.path.exists(model_path):
            print(f"  >> 오류: 모델 파일을 찾을 수 없습니다: {model_path}. 건너<0xEB><0x8A>니다.")
            continue
        loaded_model = tf.keras.models.load_model(model_path)
        print(f"  >> 모델 로드 완료: {model_path}")

        # 3.2 데이터 로드 및 준비
        if not os.path.exists(data_path):
             print(f"  >> 오류: 데이터 파일을 찾을 수 없습니다: {data_path}. 건너<0xEB><0x8A>니다.")
             continue
        df_loaded = pd.read_parquet(data_path)

        # 피처 및 타겟 추출 (학습 때와 동일하게)
        available_features = [col for col in FEATURE_COLUMNS_NORM if col in df_loaded.columns]
        if len(available_features) == 0 or TARGET_COLUMN_NORM not in df_loaded.columns:
             print(f"  >> 오류: 필요한 피처 또는 타겟 컬럼이 데이터에 없습니다. 건너<0xEB><0x8A>니다.")
             continue

        feature_data = df_loaded[available_features].values
        target_data = df_loaded[TARGET_COLUMN_NORM].values

        # NaN 재확인 및 처리 (필요시)
        if np.isnan(feature_data).any() or np.isnan(target_data).any():
             print(f"  >> 경고: 피처 또는 타겟 데이터에 NaN 존재. nan_to_num 처리.")
             feature_data = np.nan_to_num(feature_data, nan=0.0)
             target_data = np.nan_to_num(target_data, nan=0.0)

        # 3.3 시퀀스 생성 및 데이터 분할 (학습 때와 정확히 동일한 로직)
        X, y = create_sequences(feature_data, target_data, TIME_STEPS)
        if len(X) < (TIME_STEPS * 2):
             print(f"  >> 오류: 시퀀스 길이 부족. 건너<0xEB><0x8A>니다.")
             continue

        n_total = len(X)
        n_train = int(n_total * TRAIN_RATIO)
        n_valid = int(n_total * VALID_RATIO)
        # 테스트셋 인덱스: n_train + n_valid 부터 끝까지
        X_test, y_test = X[n_train + n_valid:], y[n_train + n_valid:]

        if len(X_test) == 0:
            print(f"  >> 경고: 테스트 데이터셋이 비어있습니다. 건너<0xEB><0x8A>니다.")
            continue

        print(f"  >> 테스트 데이터 준비 완료: {len(X_test)}개 시퀀스")

        # 3.4 테스트셋 평가
        test_loss = loaded_model.evaluate(X_test, y_test, verbose=0)
        print(f"  >> 테스트셋 손실 (MSE): {test_loss:.6f}")
        results[ticker] = test_loss

        # 메모리 정리
        del df_loaded, feature_data, target_data, X, y, X_test, y_test, loaded_model
        tf.keras.backend.clear_session()
        gc.collect()

    except Exception as e:
        print(f"  >> 오류: 티커 {ticker} 평가 중 예외 발생: {e}")
        import traceback
        traceback.print_exc()
        results[ticker] = 'Error'
        continue

print("\n--- 테스트셋 평가 결과 요약 ---")
for ticker, loss in results.items():
    print(f"Ticker: {ticker}, Test MSE: {loss}")

--- 5개 종목 모델 테스트셋 평가 시작 ---

[1/5] Ticker: 000020 평가 시작...
  >> 모델 로드 완료: /content/drive/MyDrive/lstm_models_per_stock_v1/000020.keras
  >> 테스트 데이터 준비 완료: 90개 시퀀스
  >> 테스트셋 손실 (MSE): 0.027195

[2/5] Ticker: 000040 평가 시작...
  >> 모델 로드 완료: /content/drive/MyDrive/lstm_models_per_stock_v1/000040.keras
  >> 테스트 데이터 준비 완료: 88개 시퀀스
  >> 테스트셋 손실 (MSE): 0.003904

[3/5] Ticker: 000050 평가 시작...
  >> 모델 로드 완료: /content/drive/MyDrive/lstm_models_per_stock_v1/000050.keras
  >> 테스트 데이터 준비 완료: 90개 시퀀스
  >> 테스트셋 손실 (MSE): 0.001835

[4/5] Ticker: 000070 평가 시작...
  >> 모델 로드 완료: /content/drive/MyDrive/lstm_models_per_stock_v1/000070.keras
  >> 테스트 데이터 준비 완료: 90개 시퀀스


  >> 테스트셋 손실 (MSE): 0.013285

[5/5] Ticker: 000075 평가 시작...
  >> 모델 로드 완료: /content/drive/MyDrive/lstm_models_per_stock_v1/000075.keras
  >> 테스트 데이터 준비 완료: 90개 시퀀스


  >> 테스트셋 손실 (MSE): 0.008861

--- 테스트셋 평가 결과 요약 ---
Ticker: 000020, Test MSE: 0.02719520777463913
Ticker: 000040, Test MSE: 0.0039039291441440582
Ticker: 000050, Test MSE: 0.0018347493605688214
Ticker: 000070, Test MSE: 0.013284851796925068
Ticker: 000075, Test MSE: 0.008861063979566097


In [ ]:
#!pip install backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 14.5 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
# backtesting.py 라이브러리 설치 필요: pip install backtesting

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
# from backtesting.test import SMA # 내장 SMA 대신 직접 계산 또는 예측값 활용

# --- 1. 설정값 ---
TICKER_FOR_BACKTEST = '000020' # 백테스팅할 티커 (위에서 평가한 것 중 하나)
DRIVE_DATA_PATH = '/content/drive/MyDrive/processed_stock_data_full_v1/processed_parquet'
DRIVE_MODEL_SAVE_PATH = '/content/drive/MyDrive/lstm_models_per_stock_v1'

# 백테스팅 설정
INITIAL_CASH = 10_000_000 # 초기 자본금
COMMISSION_RATE = 0.00015 # 수수료 (예: 0.015%)

# 이전 학습/평가와 동일한 설정값 (데이터 분할 및 시퀀스 생성용)
TIME_STEPS = 20
TRAIN_RATIO = 0.7
VALID_RATIO = 0.15
FEATURE_COLUMNS_NORM = [ # 테스트셋 예측 위해 필요
    'open_norm', 'high_norm', 'low_norm', 'close_norm', 'volume_norm', 'amount_norm',
    'SMA_5_norm', 'SMA_20_norm', 'SMA_60_norm', 'SMA_120_norm', 'RSI_14_norm',
    'MACD_12_26_9_norm', 'MACDs_12_26_9_norm', 'MACDh_12_26_9_norm',
    'BBL_20_2.0_norm', 'BBM_20_2.0_norm', 'BBU_20_2.0_norm', 'BBB_20_2.0_norm', 'BBP_20_2.0_norm',
    'ATRr_14_norm', 'OBV_norm', 'STOCHk_14_3_3_norm', 'STOCHd_14_3_3_norm',
    'PBR_norm', 'PER_norm', 'USD_KRW_norm', 'is_month_end'
]
TARGET_COLUMN_NORM = 'close_norm'

print(f"--- Ticker: {TICKER_FOR_BACKTEST} 백테스팅 시작 ---")

# --- 2. 데이터 및 모델 로드, 예측값 생성 ---
model_path = os.path.join(DRIVE_MODEL_SAVE_PATH, f"{TICKER_FOR_BACKTEST}.keras")
data_path = os.path.join(DRIVE_DATA_PATH, f"{TICKER_FOR_BACKTEST}.parquet")

try:
    # 모델 로드
    if not os.path.exists(model_path): raise FileNotFoundError("모델 파일 없음")
    model = tf.keras.models.load_model(model_path)
    print("  >> 모델 로드 완료.")

    # 데이터 로드
    if not os.path.exists(data_path): raise FileNotFoundError("데이터 파일 없음")
    df_full = pd.read_parquet(data_path)
    print(f"  >> 데이터 로드 완료: {df_full.shape}")

    # backtesting.py 입력 형식에 맞게 컬럼 선택 및 이름 변경
    # 원본 OHLCV 컬럼 사용 ('Open', 'High', 'Low', 'Close', 'Volume' 이름으로 변경)
    df_ohlcv_orig = df_full[['open', 'high', 'low', 'close', 'volume']].copy()
    df_ohlcv_orig.columns = ['Open', 'High', 'Low', 'Close', 'Volume'] # 대문자로 시작하는 이름 사용

    # 테스트 기간에 대한 예측값 미리 생성 (Lookahead bias 가능성 있음 - 주의)
    # 필요한 피처 준비
    available_features = [col for col in FEATURE_COLUMNS_NORM if col in df_full.columns]
    if len(available_features) == 0 or TARGET_COLUMN_NORM not in df_full.columns:
         raise ValueError("예측에 필요한 피처 또는 타겟 컬럼 부족")

    feature_data = df_full[available_features].values
    target_data = df_full[TARGET_COLUMN_NORM].values
    feature_data = np.nan_to_num(feature_data, nan=0.0)
    target_data = np.nan_to_num(target_data, nan=0.0)

    # 시퀀스 생성 및 분할 (테스트셋 인덱스 얻기 위함)
    def create_sequences(data, target, time_steps=10):
        Xs, ys = [], []
        feature_data = data
        for i in range(len(feature_data) - time_steps):
            Xs.append(feature_data[i:(i + time_steps)])
            ys.append(target[i + time_steps])
        return np.array(Xs), np.array(ys)

    X, y = create_sequences(feature_data, target_data, TIME_STEPS)
    n_total = len(X)
    n_train = int(n_total * TRAIN_RATIO)
    n_valid = int(n_total * VALID_RATIO)
    test_start_index_in_sequence = n_train + n_valid # 시퀀스 기준 테스트 시작 인덱스
    X_test = X[test_start_index_in_sequence:]

    # 테스트셋 기간에 대한 예측 수행
    y_pred_test_norm = model.predict(X_test).flatten() # flatten()으로 1D 배열로 만듦

    # 예측값을 원본 데이터프레임의 인덱스와 맞추기
    # 예측값은 test_start_index_in_sequence + TIME_STEPS 시점부터의 결과임
    test_data_start_index_in_df = test_start_index_in_sequence + TIME_STEPS
    test_pred_dates = df_full.index[test_data_start_index_in_df:]

    # 예측 길이나 날짜 길이 불일치 시 조정 (y_pred_test_norm 길이 기준으로 자름)
    min_len = min(len(test_pred_dates), len(y_pred_test_norm))
    test_pred_dates = test_pred_dates[:min_len]
    y_pred_test_norm = y_pred_test_norm[:min_len]

    # 예측값을 Series로 만들어 원본 OHLCV 데이터에 추가 (날짜 인덱스 기준)
    predictions = pd.Series(y_pred_test_norm, index=test_pred_dates, name='Prediction')
    df_backtest_data = pd.merge(df_ohlcv_orig, predictions, left_index=True, right_index=True, how='left')
    # backtesting 라이브러리는 NaN 예측값을 처리하지 못하므로, 예측값이 없는 앞부분은 제거
    df_backtest_data.dropna(subset=['Prediction'], inplace=True)

    print(f"  >> 백테스팅용 데이터 준비 완료 (예측값 포함): {df_backtest_data.shape}")
    if df_backtest_data.empty:
        raise ValueError("백테스팅용 데이터가 비어 있습니다.")

except Exception as e:
    print(f"데이터/모델 준비 중 오류: {e}")
    exit()

# --- 3. 백테스팅 전략 정의 ---
class LstmSignalStrategy(Strategy):
    # (옵션) 전략 파라미터 정의
    threshold = 0.005 # 예측값이 현재 종가(정규화된 값 기준)보다 0.5% 높거나 낮을 때 진입

    def init(self):
        # 예측값은 외부에서 계산되어 데이터프레임에 포함됨 ('Prediction' 컬럼)
        self.prediction = self.data.Prediction # Prediction 시리즈 가져오기

        # (참고) 현재 종가의 정규화된 값도 필요함 (비교 대상)
        # 이를 위해 원본 데이터나 스케일러 필요. 여기서는 단순화 위해 예측값만 사용
        # 좀 더 정확하려면 스케일러 로드 후 현재 close값을 정규화하여 비교해야 함

    def next(self):
        # 현재 가격 (Close)
        current_price = self.data.Close[-1]
        # 다음 날 예측 가격 (정규화된 값) - init에서 가져옴
        # self.I()를 사용하면 backtesting이 자동으로 관리하나, 외부 예측값 사용 시 직접 인덱싱
        # self.data.index[-1]은 현재 날짜, 예측값은 다음날에 대한 것이므로 주의 필요
        # 여기서는 가장 최근 예측값을 사용한다고 가정 (단순화)
        predicted_norm = self.prediction[-1] # 가장 최근 예측값

        # (개선 필요) 아래는 매우 단순화된 로직. 예측값의 절대 레벨이 아닌,
        # 예측된 '변화 방향' 이나 '변화율'을 사용하는 것이 더 일반적임.
        # 또한, 현재가(정규화) 대비 예측가(정규화) 비교가 더 타당함.

        # 예시: 예측값이 현재 값보다 threshold만큼 높으면 매수, 낮으면 매도 (매우 단순)
        # 실제로는 예측 변화율 등을 사용해야 함

        # (매우 단순한 예시 로직 - 실제 사용 부적합)
        # if not self.position: # 포지션이 없으면
        #     if predicted_norm > 0.6: # 예: 정규화된 예측값이 0.6보다 크면 매수 시도
        #         self.buy()
        # elif self.position.is_long: # 매수 포지션이 있으면
        #      if predicted_norm < 0.4: # 예: 정규화된 예측값이 0.4보다 작으면 매도
        #          self.position.close()

        # 좀 더 나은 예시: 예측된 '방향성' 사용
        # 예측값이 '현재 값' (정규화된) 보다 높을 것으로 예상되면 매수 고려
        # 이를 위해서는 현재 값의 정규화 필요. 여기서는 임시로 이전 예측값과 비교
        if len(self.prediction) >= 2:
            last_prediction = self.prediction[-2]
            current_prediction = self.prediction[-1]

            if not self.position: # 포지션 없으면
                if current_prediction > last_prediction * (1 + self.threshold): # 상승 예측 시 매수
                    self.buy()
            elif self.position.is_long: # 매수 포지션 있으면
                 if current_prediction < last_prediction * (1 - self.threshold): # 하락 예측 시 매도
                     self.position.close()


# --- 4. 백테스팅 실행 ---
bt = Backtest(df_backtest_data, LstmSignalStrategy,
              cash=INITIAL_CASH, commission=COMMISSION_RATE)

stats = bt.run()

print("\n--- 백테스팅 결과 ---")
print(stats)

# --- 5. 결과 시각화 ---
print("\n--- 백테스팅 결과 시각화 ---")
# Colab 등 환경에서 차트 표시 위해 필요할 수 있음
# %matplotlib inline
bt.plot()

--- Ticker: 000020 백테스팅 시작 ---
  >> 모델 로드 완료.
  >> 데이터 로드 완료: (616, 54)
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 394ms/step
  >> 백테스팅용 데이터 준비 완료 (예측값 포함): (90, 6)


Backtest.run:   0%|          | 0/89 [00:00<?, ?bar/s]


--- 백테스팅 결과 ---
Start                     2024-08-16 00:00:00
End                       2024-12-30 00:00:00
Duration                    136 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   7377701.23
Equity Peak [$]                    10000000.0
Return [%]                          -26.22299
Buy & Hold Return [%]               -24.09639
Return (Ann.) [%]                   -57.32446
Volatility (Ann.) [%]                 9.31985
CAGR [%]                            -43.07992
Sharpe Ratio                         -6.15079
Sortino Ratio                        -3.13767
Calmar Ratio                         -1.91292
Alpha [%]                            -3.19399
Beta                                   0.9557
Max. Drawdown [%]                   -29.96699
Avg. Drawdown [%]                   -29.96699
Max. Drawdown Duration      133 days 00:00:00
Avg. Drawdown Duration      133 days 00:00:00
# Trades                                    0
Win Rate [%]     

GridPlot(id='p1157', ...)